In [7]:
import torch
import numpy as np
import pandas as pd
import scipy.io as scio
from torch.utils.data import Dataset,DataLoader
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import random

In [8]:
class MyDataset(Dataset):
        def __init__(self,data,topk):
            self.data=data
            self.topk=topk
            self.label={
            'T-T':2,
            'P-T':1,
            'P-P':0
            }

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            path_1,label=self.data[idx]
            image=scio.loadmat(path_1)['fc']
            image=torch.from_numpy(image).float()
            label=torch.tensor(self.label[label])
            return image,label


In [9]:
X = []  # 定义图像名称
Y = []  # 定义图像分类类标

with open('fc_list.txt','r') as f:
    data=f.readlines()
data=[item.replace('\n','').split('\t') for item in data]
random.seed(2022)
random.shuffle(data)
X=[x[0] for x in data]
Y=[x[1] for x in data]

X = np.array(X)
Y = np.array(Y)

# 随机率为100% 选取其中的30%作为测试集
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)
# print(len(X_train), len(X_test), len(y_train), len(y_test))

XX_train = []
for imagepath in X_train:
    # temp_image=h5py.File(imagepath,'r')
    temp_image=scio.loadmat(imagepath)['fc']
    XX_train.append(temp_image.flatten())  #reshape into 1D array

# 测试集
XX_test = []
for imagepath in X_test:
    # temp_image=h5py.File(imagepath,'r')
    temp_image=scio.loadmat(imagepath)['fc']
    XX_test.append(temp_image.flatten())

In [10]:
result=[]
from sklearn.metrics import confusion_matrix
result_index_matrix=torch.zeros(5,3,3)
for round in range(5):
    dtModel = DecisionTreeClassifier()  # 建立模型
    dtModel.fit(XX_train,y_train)  
    prediction = dtModel.predict(XX_test)  

    c_matrix=confusion_matrix(y_test,prediction)
    for idm in range(3):
        result_index_matrix[round][0][idm]=c_matrix[idm][idm]/np.sum(c_matrix[:,idm])
        result_index_matrix[round][1][idm]=c_matrix[idm][idm]/np.sum(c_matrix[idm,:])
        result_index_matrix[round][2][idm]=(np.sum(c_matrix)-np.sum(c_matrix[:,idm])-np.sum(c_matrix[idm,:])+c_matrix[idm][idm])/(np.sum(c_matrix)-np.sum(c_matrix[idm,:])) 
       
    acc = accuracy_score(y_test,prediction)  
    print(acc)
    print(result_index_matrix[round])

    result.append(acc)
print('mean:{:.4f}  std:{:.4f}'.format(np.mean(result),np.std(result)))
mean_matrix=result_index_matrix.sum(dim=0)/5
print(mean_matrix)
print(mean_matrix.sum(dim=1)/3)

0.7705627705627706
tensor([[0.6364, 0.7727, 0.9221],
        [0.8116, 0.6145, 0.8987],
        [0.8025, 0.8986, 0.9605]])
0.7878787878787878
tensor([[0.6914, 0.7500, 0.9231],
        [0.8116, 0.6506, 0.9114],
        [0.8457, 0.8784, 0.9605]])
0.8008658008658008
tensor([[0.6914, 0.7778, 0.9359],
        [0.8116, 0.6747, 0.9241],
        [0.8457, 0.8919, 0.9671]])
0.7619047619047619
tensor([[0.6429, 0.7576, 0.8889],
        [0.7826, 0.6024, 0.9114],
        [0.8148, 0.8919, 0.9408]])
0.7878787878787878
tensor([[0.6747, 0.7714, 0.9231],
        [0.8116, 0.6506, 0.9114],
        [0.8333, 0.8919, 0.9605]])
mean:0.7818  std:0.0139
tensor([[0.6673, 0.7659, 0.9186],
        [0.8058, 0.6386, 0.9114],
        [0.8284, 0.8905, 0.9579]])
tensor([0.7839, 0.7852, 0.8923])
